In [ ]:
import typing
import qokit
import numpy as np
import scipy
import networkx as nx
import time
from matplotlib import pyplot as plt
from qokit.fur.qaoa_simulator_base import QAOAFastSimulatorBase, TermsType
import qokit.maxcut as mc

In [ ]:
def compute_graph_colors(G: nx.Graph, solution_index: int) -> tuple[list[str], list[str]]:
    N = G.number_of_nodes()
    vert_colors = list(map(lambda c: "green" if c == "1" else "blue", f"{int(solution_index):0{N}b}"))
    vert_colors.reverse()
    edge_colors = [("red" if vert_colors[u] != vert_colors[v] else "black") for u, v in G.edges()]
    return (vert_colors, edge_colors)


def print_result(N: int, terms: TermsType, gamma: np.ndarray, beta: np.ndarray, adjective: str = "") -> None:
    p = len(gamma)
    # print(f"{adjective}Gamma: {gamma}")
    # print(f"{adjective}Beta: {beta}")
    # print(f"{adjective}Objective: {-inv_max_cut_objective(N, p, terms)(np.hstack([gamma, beta]))}")
    # print()


def evaluate_cut(G: nx.Graph, bool_verts: list[bool]) -> int:
    cuts = 0
    for u, v in G.edges():
        if bool_verts[u] != bool_verts[v]:
            cuts += 1
    return cuts


def print_probabilities(
    G: nx.Graph, terms: TermsType, gamma: np.ndarray, beta: np.ndarray, threshold: float = 0, only_likely: bool = False, include_cost: bool = True
) -> None:
    N = G.number_of_nodes()
    probs = get_probabilities(N, terms, gamma, beta)
    for element in np.column_stack((np.array(range(2**N)), probs)):
        [index, prob] = element
        likely_str = ""
        if prob > threshold:
            likely_str = " <- Likely outcome"

        cost_str = ""
        if include_cost:
            contained_verts = list(map(lambda c: c == "1", f"{int(index):0{N}b}"))
            contained_verts.reverse()
            cut_count = evaluate_cut(G, contained_verts)
            cost_str = f" ({cut_count})"

        # if not only_likely:
        #     print(f"{int(index):0{N}b}{cost_str} - {prob}{likely_str}")
        # else:
        #     if prob > threshold:
        #         print(f"{int(index):0{N}b}{cost_str} - {prob}")


def print_state_vector(G: nx.Graph, terms: TermsType, gamma: np.ndarray, beta: np.ndarray, include_cost: bool = True) -> None:
    N = G.number_of_nodes()
    sv = get_state_vector(N, terms, gamma, beta)
    for element in np.column_stack((np.array(range(2**N)), sv)):
        [index, amplitude] = element
        cost_str = ""
        if include_cost:
            contained_verts = list(map(lambda c: c == "1", f"{int(index):0{N}b}"))
            contained_verts.reverse()
            cut_count = evaluate_cut(G, contained_verts)
            cost_str = f" ({cut_count})"

        # print(f"{int(index):0{N}b}{cost_str} - {amplitude}")


def plot_expectation(expectations: list[float], N: int, p: int, start_time: float) -> None:
    def make_time_relative(input: tuple[float, float]) -> tuple[float, float]:
        time, expectation = input
        return (time - start_time, expectation)

    time_relative_expectations = list(map(make_time_relative, expectations))
    # plt.scatter(*zip(*time_relative_expectations))
    # plt.title(f"MaxCut (N = {N}, p = {p})")
    # plt.xlabel("Time (seconds)")
    # plt.ylabel("QAOA Expectation")
    # plt.show()


def plot_measurements(measurements: list[float], N: int, p: int, start_time: float) -> None:
    def make_time_relative(input: tuple[float, float]) -> tuple[float, float]:
        time, measurement = input
        return (time - start_time, measurement)

    time_relative_measurements = list(map(make_time_relative, measurements))
    # plt.scatter(*zip(*time_relative_measurements))
    # plt.title(f"MaxCut (N = {N}, p = {p})")
    # plt.xlabel("Time (seconds)")
    # plt.ylabel("Measurement (best of 10)")
    # plt.show()


def optimal_solution_with_index(sim: QAOAFastSimulatorBase) -> tuple[float, int]:
    costs = sim.get_cost_diagonal()
    solution = max(costs)
    solution_index = np.where(costs == solution)[0][0]
    return (solution, solution_index)


def get_simulator(N: int, terms: TermsType, sim_or_none: QAOAFastSimulatorBase | None = None) -> QAOAFastSimulatorBase:
    if sim_or_none is None:
        simclass = qokit.fur.choose_simulator(name="auto")
        return simclass(N, terms=terms)
    else:
        return sim_or_none


def get_result(
    N: int, terms: TermsType, gamma: np.ndarray, beta: np.ndarray, sim: QAOAFastSimulatorBase | None = None, result: np.ndarray | None = None
) -> np.ndarray:
    if result is None:
        simulator = get_simulator(N, terms, sim)
        return simulator.simulate_qaoa(gamma, beta)
    else:
        return result


def get_simulator_and_result(
    N: int, terms: TermsType, gamma: np.ndarray, beta: np.ndarray, sim: QAOAFastSimulatorBase | None = None, result: np.ndarray | None = None
) -> tuple[QAOAFastSimulatorBase, np.ndarray]:
    simulator = get_simulator(N, terms, sim)
    if result is None:
        result = get_result(N, terms, gamma, beta, simulator)
    return (simulator, result)


def get_probabilities(
    N: int, terms: TermsType, gamma: np.ndarray, beta: np.ndarray, sim: QAOAFastSimulatorBase | None = None, result: np.ndarray | None = None
) -> np.ndarray:
    simulator, result = get_simulator_and_result(N, terms, gamma, beta, sim, result)
    return simulator.get_probabilities(result, preserve_state=True)


def get_expectation(
    N: int, terms: TermsType, gamma: np.ndarray, beta: np.ndarray, sim: QAOAFastSimulatorBase | None = None, result: np.ndarray | None = None
) -> float:
    simulator, result = get_simulator_and_result(N, terms, gamma, beta, sim, result)
    return simulator.get_expectation(result, preserve_state=True)


def get_state_vector(N: int, terms: TermsType, gamma: np.ndarray, beta: np.ndarray) -> np.ndarray:
    simclass = qokit.fur.choose_simulator(name="auto")
    sim = simclass(N, terms=terms)
    _result = sim.simulate_qaoa(gamma, beta)
    return sim.get_statevector(_result)


# Evaluates the objective function, and if expectations or measurements is passed in, appends to them
def inv_max_cut_objective(
    N: int, p: int, terms: TermsType, expectations: list[float] | None = None, measurements: list[float] | None = None, sim: QAOAFastSimulatorBase | None = None
) -> typing.Callable:
    def f(*args) -> float:
        gamma, beta = args[0][:p], args[0][p:]
        current_time = time.time()
        simulator = get_simulator(N, terms, sim)
        probs = get_probabilities(N, terms, gamma, beta, sim)
        costs = simulator.get_cost_diagonal()
        expectation = np.dot(costs, probs)

        if expectations != None:
            expectations.append((current_time, expectation))

        if measurements != None:
            measurement = max(np.random.choice(costs, 10, p=probs))
            measurements.append((current_time, measurement))

        return -expectation

    return f


# Given Eising terms and initial parameters, runs scipy.optimize.minimize and returns optimized parameters
# If expectations or measurements is passed in, populates them (this harms runtime)
def optimize(
    N: int,
    terms: TermsType,
    init_gamma: np.ndarray,
    init_beta: np.ndarray,
    expectations: list[float] | None = None,
    measurements: list[float] | None = None,
    sim: QAOAFastSimulatorBase | None = None,
) -> tuple[np.ndarray, np.ndarray]:
    if expectations != None:
        expectations.clear()
    if measurements != None:
        measurements.clear()

    p = len(init_gamma)
    assert len(init_beta) == p, "Gamma and Beta must have the same length"
    init_freq = np.hstack([init_gamma, init_beta])

    res = scipy.optimize.minimize(inv_max_cut_objective(N, p, terms, expectations, measurements, sim), init_freq, method="COBYLA", options={"rhobeg": 0.01 / N})

    gamma, beta = res.x[:p], res.x[p:]
    return (gamma, beta)


# Generates a random graph with N vertices and runs MaxCut QAOA on this graph, plotting results
def run_experiment(seed: int, N: int, p: int) -> None:
    np.random.seed(seed)
    G = nx.erdos_renyi_graph(N, 0.5, seed=seed)  # Random graph w/ 0.5 edge probability
    terms = mc.get_maxcut_terms(G)
    sim = get_simulator(N, terms)
    expectations = []
    measurements = []

    # Compute and print solution by brute force
    solution, solution_index = optimal_solution_with_index(sim)
    # print(f"Optimal Solution: {solution_index:0{N}b} - {solution}")
    vert_colors, edge_colors = compute_graph_colors(G, solution_index)

    # Random starting parameters
    init_gamma, init_beta = np.random.rand(2, p)
    # print_result(N, terms, init_gamma, init_beta, "Initial ")

    start_time = time.time()
    # If you don't want expectation and measurement plots, do not pass in parameters!
    gamma, beta = optimize(N, terms, init_gamma, init_beta, sim=sim)
    end_time = time.time()
    # print(f"Time to optimize: {end_time - start_time} seconds\n")

    # print_result(N, terms, gamma, beta, "Final ")
    # print_probabilities(G, terms, gamma, beta, threshold=0.5 / N, only_likely=True)
    # print()
    # plot_expectation(expectations, N, p, start_time)
    # plot_measurements(measurements, N, p, start_time)

    # nx.draw(G, nx.circular_layout(G), node_color=vert_colors, edge_color=edge_colors, with_labels=True)
    return (end_time - start_time)

# List of Ns
Ps = list(range(1, 11))
runtimes = []

# Run experiments for each N and collect runtimes
for i in Ps:
    runtime = run_experiment(seed = 0, N = 10, p = i)
    runtimes.append(runtime)
    print(f"p = {i}: Time to optimize = {runtime} seconds")

# Plotting runtime vs P
plt.plot(Ps, runtimes, marker='o')
plt.title('Runtime vs P')
plt.xlabel('P (Circuit Depth)')
plt.ylabel('Runtime (seconds)')
plt.grid(True)
plt.show()